# Mix Self-Consistency Notebook 

<a href="https://colab.research.google.com/github/run-llama/llama-hub/blob/main/llama_hub/llama_packs/tables/mix_self_consistency/mix_self_consistency.ipynb" target="_parent">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we highlight the mix self-consistency method proposed in "Rethinking Tabular Data Understanding with Large Language Models" paper by Liu et al.[https://arxiv.org/pdf/2312.16702v1.pdf].

LLMs can reason over tabular data in 2 main ways:
1. textual reasoning via direct prompting
2. symbolic reasoning via program synthesis (e.g. python, SQL, etc)

The key insight of the paper is that different reasoning pathways work well in different tasks. By aggregating results from both with a self-consistency mechanism (i.e. majority voting), it achieves SoTA performance.

We implemented the paper based on the prompts described in the paper, and adapted it to get it working. That said, this is marked as beta, so there may still be kinks to work through. Do you have suggestions / contributions on how to improve the robustness? Let us know! 

# Download Data

We use the [WikiTableQuestions dataset](https://ppasupat.github.io/WikiTableQuestions/) (Pasupat and Liang 2015) as our test dataset.

WikiTableQuestions is a question-answering dataset over various semi-structured tables taken from Wikipedia. These tables range in size from a few rows/columns to mnay rows. Some columns may contain multi-part information as well (e.g. a temperature column may contain both Fahrenheight and Celsius).

In [ ]:
!wget "https://github.com/ppasupat/WikiTableQuestions/releases/download/v1.0.2/WikiTableQuestions-1.0.2-compact.zip" -O data.zip
!unzip data.zip

Let's visualize some examples.

In [1]:
import pandas as pd

examples = pd.read_table('WikiTableQuestions/data/training-before300.tsv')

examples.head()

,id,utterance,context,targetValue
0,nt-0,what was the last year where this team was a p...,csv/204-csv/590.csv,2004
1,nt-1,in what city did piotr's last 1st place finish...,csv/204-csv/622.csv,"Bangkok, Thailand"
2,nt-2,which team won previous to crettyard?,csv/204-csv/772.csv,Wolfe Tones
3,nt-3,how many more passengers flew to los angeles t...,csv/203-csv/515.csv,"12,467"
4,nt-4,who was the opponent in the first game of the ...,csv/204-csv/495.csv,Derby County


In [2]:
example = examples.iloc[0]
question = example['utterance']
context = example['context']
print('The question is: ', question)

The question is:  what was the last year where this team was a part of the usl a-league?


Let's load the table that can be used as context to answer the question in the first example.

In [3]:
table = pd.read_csv('WikiTableQuestions/' + context)
table.head()

,Year,Division,League,Regular Season,Playoffs,Open Cup,Avg. Attendance
0,2001,2,USL A-League,"4th, Western",Quarterfinals,Did not qualify,"7,169"
1,2002,2,USL A-League,"2nd, Pacific",1st Round,Did not qualify,"6,260"
2,2003,2,USL A-League,"3rd, Pacific",Did not qualify,Did not qualify,"5,871"
3,2004,2,USL A-League,"1st, Western",Quarterfinals,4th Round,"5,628"
4,2005,2,USL First Division,5th,Quarterfinals,4th Round,"6,028"


The corect answer should be 2004.

## Load Pack / Setup

Now we do `download_llama_pack` to load the Mix Self Consistency LlamaPack (you can also import the module directly if using the llama-hub package).

We will also optionally setup observability/tracing so we can observe the intermediate steps.

In [4]:
# Option: if developing with the llama_hub package
from llama_hub.llama_packs.tables.mix_self_consistency.base import (
    MixSelfConsistencyQueryEngine,
)

# Option: download llama_pack
# from llama_index.llama_pack import download_llama_pack

# download_llama_pack(
#     "MixSelfConsistencyPack",
#     "./mix_self_consistency_pack",
#     skip_load=True,
#     # leave the below line commented out if using the notebook on main
#     llama_hub_url="https://raw.githubusercontent.com/run-llama/llama-hub/suo/table_qa/llama_hub"
# )
# from mix_self_consistency_pack.base import MixSelfConsistencyQueryEngine

In [5]:
from llama_index.llms import OpenAI

llm = OpenAI()

### Optional: Setup Observability

Here we will use our Arize Phoenix integration to view traces through the query engine.

In [ ]:
import phoenix as px
import llama_index

px.launch_app()
llama_index.set_global_handler("arize_phoenix")

## Run experiments

Let's try out different modes.

### Textual Reasoning Only (i.e. direct prompting)

Let's start with using only the textual reasoning path.
Basically, we directly convert the pandas dataframe into a markdown representation, and inject into the prompt for in-context reasoning.

In [7]:
query_engine = MixSelfConsistencyQueryEngine(
    table=table, 
    llm=llm,
    text_paths=1,
    symbolic_paths=0,
    aggregation_mode='none',
    verbose=True,
)

In [8]:
response = await query_engine.aquery(example['utterance'])
print(response)

> Running module c6ffdfda-15f3-4a4c-9d7d-3a7ab025437c with input: 
question: what was the last year where this team was a part of the usl a-league?
table: |    |   Year |   Division | League              | Regular Season   | Playoffs        | Open Cup        | Avg. Attendance   |
|---:|-------:|-----------:|:--------------------|:-----------------|:----...

> Running module fa48cf7a-e2c2-47d7-994c-2e631e9c84b0 with input: 
messages: You are an advanced AI capable of analyzing and understanding information within tables. Read the table below.

|    |   Year |   Division | League              | Regular Season   | Playoffs        | ...

> Running module df320e1c-8268-4f9f-813d-4ce0d9322de6 with input: 
input: assistant: Step 1: Identify the rows where the team was a part of the USL A-League.
- Row 0: Year 2001, Division 2, League USL A-League
- Row 1: Year 2002, Division 2, League USL A-League
- Row 2: Yea...

Text results: ['2003']
Symbolic results: []
2003


We get an incorrect answer. 

In [9]:
query_engine = MixSelfConsistencyQueryEngine(
    table=table, 
    llm=llm,
    text_paths=5,
    symbolic_paths=0,
    aggregation_mode='self_consistency',
    verbose=True,
)

In [10]:
response = await query_engine.aquery(example['utterance'])
print(response)

> Running module 09c4945f-0eee-4cf6-b33c-82145ad6b13e with input: 
question: what was the last year where this team was a part of the usl a-league?
table: |    |   Year |   Division | League              | Regular Season   | Playoffs        | Open Cup        | Avg. Attendance   |
|---:|-------:|-----------:|:--------------------|:-----------------|:----...

> Running module 9d579716-5a76-4da2-97be-00daf3ac78df with input: 
messages: You are an advanced AI capable of analyzing and understanding information within tables. Read the table below.

|    |   Year |   Division | League              | Regular Season   | Playoffs        | ...

> Running module c7954d0c-d4a4-4034-a3f1-3207d55f9c7b with input: 
question: what was the last year where this team was a part of the usl a-league?
table: |    |   Year |   Division | League              | Regular Season   | Playoffs        | Open Cup        | Avg. Attendance   |
|---:|-------:|-----------:|:--------------------|:-----------------|:----...

We still get incorrect result after sampling 5 textua reasoning paths, and aggregating the results via self-consistency.

### Symbolid Reasoning Only (i.e. python shell)

Now, let's use symbolic reasoning. Here, LLM generates python expressions directly manipuate the pandas dataframe.

In [13]:
query_engine = MixSelfConsistencyQueryEngine(
    table=table, 
    llm=llm,
    text_paths=0,
    symbolic_paths=1,
    aggregation_mode='none',
    verbose=True,
)

In [14]:
response = await query_engine.aquery(example['utterance'])
print(response)

> Pandas Instructions:
```
df[df['League'] == 'USL A-League']['Year'].max()
```
> Pandas Output: 2004
Text results: []
Symbolic results: ['2004']
2004


We get the correct answer here. 

### Aggregation via Self-Evaluation

Now we consider self-evaluation for aggregating results across textual and symbolic reasoning paths.
Basically, we tell the LLM what each reasoning path is good at to obtain a final result.

In [15]:
query_engine = MixSelfConsistencyQueryEngine(
    table=table, 
    llm=llm,
    text_paths=1,
    symbolic_paths=1,
    aggregation_mode='self_evaluation',
    verbose=True,
)

In [16]:
response = await query_engine.aquery(example['utterance'])
print(response)

> Running module 159fc440-4a11-472b-860c-c0e83c11eb97 with input: 
question: what was the last year where this team was a part of the usl a-league?
table: |    |   Year |   Division | League              | Regular Season   | Playoffs        | Open Cup        | Avg. Attendance   |
|---:|-------:|-----------:|:--------------------|:-----------------|:----...

> Running module 2666e967-c255-4c55-a157-978e1dd2c2ec with input: 
messages: You are an advanced AI capable of analyzing and understanding information within tables. Read the table below.

|    |   Year |   Division | League              | Regular Season   | Playoffs        | ...

> Pandas Instructions:
```
df[df['League'] == 'USL A-League']['Year'].max()
```
> Pandas Output: 2004
> Running module 74cbb743-8519-4454-a3a8-125e4a6ec4da with input: 
input: assistant: Step 1: Identify the rows where the team was a part of the USL A-League.
Rows 0, 1, 2, 3 indicate that the team was a part of the USL A-League.

Step 2: Determine the last

We obtain the correct result now.

### Aggregation via Mix Self-Consistency

Now, we consider the SoTA method, which aggregates results across reasoning paths via self-consistency (i.e. majority voting) 

In [19]:
query_engine = MixSelfConsistencyQueryEngine(
    table=table, 
    llm=llm,
    text_paths=5,
    symbolic_paths=5,
    aggregation_mode='self_consistency',
    verbose=True,
)

In [20]:
response = await query_engine.aquery(example['utterance'])
print(response)

> Running module 0c3ddd36-8afe-448c-b178-13069037e735 with input: 
question: what was the last year where this team was a part of the usl a-league?
table: |    |   Year |   Division | League              | Regular Season   | Playoffs        | Open Cup        | Avg. Attendance   |
|---:|-------:|-----------:|:--------------------|:-----------------|:----...

> Running module 89924095-f833-4ac0-a970-bf12425a6e78 with input: 
messages: You are an advanced AI capable of analyzing and understanding information within tables. Read the table below.

|    |   Year |   Division | League              | Regular Season   | Playoffs        | ...

> Running module 11c478b8-91bc-4c3e-b351-8c98a7955c7d with input: 
question: what was the last year where this team was a part of the usl a-league?
table: |    |   Year |   Division | League              | Regular Season   | Playoffs        | Open Cup        | Avg. Attendance   |
|---:|-------:|-----------:|:--------------------|:-----------------|:----...